# Machine Learning: Train Incrementally on Large Datasets

Some scikit-learn estimators implement a `partial_fit` method.
This means they can be trained incrementally. The `Incremental` meta-estimator in
Dask-ML provides a nice bridge between data stored in a Dask Array and estimators implementing `partial_fit`.

In [ ]:
from dask.distributed import Client
client = Client()
client

## Create a random dataset

We'll generate a large random dataset. In practice, you would load data from a shared file system.

In [ ]:
import dask
import dask.array as da
from distributed.utils import format_bytes

import dask_ml.datasets
import dask_ml.model_selection

In [ ]:
n_samples = 4_000_000
n_features = 1_000
chunks = n_samples // 50

X, y = dask_ml.datasets.make_classification(
    n_samples=n_samples, n_features=n_features,
    chunks=chunks,
    random_state=0
)

## Split into training and testing data

We'll split the data into train and test sets.

In [ ]:
X_train, X_test, y_train, y_test = dask_ml.model_selection.train_test_split(
    X, y
)
X_train, X_test, y_train, y_test = dask.persist(
    X_train, X_test, y_train, y_test
)

format_bytes(X_train.nbytes)

## Create a Scikit-Learn Model, SGDClassifier

We'll use an `SGDClassifier` for the underlying estimator implementing `partial_fit`.
`Incemental` will sequentially pass blocks of the dask array to it.

In [ ]:
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

estimator = SGDClassifier(
    max_iter=1000,
    random_state=0
)

inc = Incremental(estimator, scoring='accuracy')

In [ ]:
%time _ = inc.fit(X_train, y_train, classes=[0, 1])

Note that this isn't parallel, but it is very general, the `Incremental` meta-estimator helps you apply any Scikit-Learn estimator that supports the `partial_fit` API across large datasets.

## Predict

Prediction is lazy and returns a Dask Array. It's generally preferable to keep data on the cluster, rather than trying to bring large results back to a single machine.

In [ ]:
inc.predict(X_test)

Scoring is immediate, but happens on the cluster.

In [ ]:
%time inc.score(X_test, y_test)

So this model isn't particularly good, but we were able to score a large dataset quickly.